$\newcommand{\To}{\Rightarrow}$

In [1]:
# Initialize to directory holpy. Run this only once!
import os, sys
os.chdir('..')

In [2]:
from kernel.type import TFun, BoolType, NatType, IntType, RealType
from kernel import term
from kernel.term import Var, Const, Comb, Term, true, false, And, Or, Implies, Not, Eq
from data import nat
from data import real
from logic import basic

basic.load_theory('real')

## Terms

We now introduce the concept of *terms*. Terms are the building block of the language of higher-order logic. To understand types and terms intuitively, we make an analogy to ordinary programming languages: in a statement `int x = y + z`, both `x` and `y + z` are terms while `int` is a type. The statement declares a new variable `x` of type `int`, and sets it to the value of the term `y + z`. In higher-order logic, each term has a unique type (introduced in the previous section). A basic example of a term is a *variable*. It can be thought of as an arbitrary but fixed value in the current context. It is constructed by providing a name and a type:

In [3]:
a = Var("a", NatType)
print(a)

a


Note a variable is displayed using its name. Its type is not displayed. To get the type of any term, we use the `get_type()` method:

In [4]:
print(a.get_type())

nat


Another example of a term is a *constant*. A constant is assumed to represent some fixed mathematical concept. Examples of constants include the boolean values true and false, and natural numbers zero, one, etc.

In [5]:
print(Const("true", BoolType))
print(Const("false", BoolType))
print(Const("zero", NatType))
print(Const("one", NatType))

true
false
(0::nat)
(1::nat)


Note the printing of 0 and 1 with type annotations. This is in order to distinguish it from 0 and 1 as integers, real numbers, etc, which can be constructed as follows:

In [6]:
print(Const("zero", IntType))
print(Const("one", IntType))
print(Const("zero", RealType))
print(Const("one", RealType))

(0::int)
(1::int)
(0::real)
(1::real)


These are used frequently so we provide shortcuts for them:

In [7]:
print(true)
print(false)
print(nat.zero)
print(real.zero)

true
false
(0::nat)
(0::real)


A variable (or constant) can also have function type. For example, the following declares a variable that is a property of natural numbers:

In [8]:
f = Var("f", TFun(NatType, BoolType))
print(f.get_type())

nat => bool


Given a term $f$ of function type $A \To B$, and a term $a$ of type $A$, we can form the term $f\ a$, representing the evaluation of $f$ at $a$. In ordinary mathematics, this is also written as $f(a)$. However, we follow the usual convention in higher-order logic, with a space between two terms denoting function application. The term $f\ a$ can be constructed as follows (here Comb stands for *combination*):

In [9]:
c = Comb(f, a)
print(c)

f a


We take advantage of Python's ability to install custom `__call__` functions to create the following shortcut for forming combinations:

In [10]:
c = f(a)
print(c)

f a


A function application $f\ a$ is only permitted when $f$ has function type, and the type of $a$ agrees with the domain of that function type. In all other cases, the function application is illegal, and we say *type checking* fails on that term. While we can form a function application that is illegal, an error is raised when we try to *type-check* the term. Type checking is performed with `checked_get_type()`, which also returns the type of the term. Here is an example where `checked_get_type()` fails:

In [11]:
f = Var("f", TFun(NatType, BoolType))
x = Var("x", BoolType)
t = f(x)
t.checked_get_type()   # raises TypeCheckException

TypeCheckException: type mismatch in application. Expected nat. Got bool

Given a function with two arguments, we can evaluate the function one argument at a time. For example, if $g$ is a function of type $nat \To nat \To nat$, and $a$ and $b$ are both of type $nat$, then $(g\ a)\ b$ is the evaluation of $g$ on $a$ and $b$. Note this is very different from the term $g\ (a\ b)$ (which will not type-check). Since the former occurs more frequently, we have the convention that function application associates to the left. Hence, $(g\ a)\ b$ can be written more compactly as $g\ a\ b$.

In [12]:
g = Var("g", TFun(NatType, NatType, BoolType))
a = Var("a", NatType)
b = Var("b", NatType)
print(g(a)(b))

g a b


But in fact, the `__call__` function for terms is written such that all arguments can be provided at the same time:

In [13]:
print(g(a, b))

g a b


While the printed form $g\ a\ b$ is very simple, it is worth remembering how the term is represented behind the scenes. The `repr` function returns the detailed representation of the term:

In [14]:
print(repr(g(a, b)))

Comb(Comb(Var(g, nat => nat => bool), Var(a, nat)), Var(b, nat))


## Common operators

Many of the most frequently used operators are functions on two arguments. For example, addition and multiplication are both functions of type $nat \To nat \To nat$. In propositional logic, conjunction and disjunction are both functions of type $bool \To bool \To bool$. Negation is a function of type $bool \To bool$.

In [15]:
print(repr(nat.plus))
print(repr(nat.times))
print(repr(term.conj))
print(repr(term.disj))
print(repr(term.neg))

Const(plus, nat => nat => nat)
Const(times, nat => nat => nat)
Const(conj, bool => bool => bool)
Const(disj, bool => bool => bool)
Const(neg, bool => bool)


These can be applied to terms just like any function. For example, the term $x + y$ (where $x$ and $y$ are natural numbers) is represented as follows:

In [16]:
x = Var("x", NatType)
y = Var("y", NatType)
z = Var("z", NatType)
t = nat.plus(x, y)
print(t)

x + y


Note how the term is automatically printed in infix form. The printing function understands usual order of evaluation rules, with addition and multiplication associate to the left.

In [17]:
print(nat.plus(x, nat.times(y, z)))
print(nat.times(nat.plus(x, y), z))
print(nat.plus(x, nat.plus(y, z)))
print(nat.plus(nat.plus(x, y), z))

x + y * z
(x + y) * z
x + (y + z)
x + y + z


We used `nat.plus` and `nat.times` to show the construction of terms more explicitly. However, infix operators between terms are also provided.

In [18]:
print(x + y * z)
print((x + y) * z)
print(x + (y + z))
print((x + y) + z)

x + y * z
(x + y) * z
x + (y + z)
x + y + z


Likewise, we can use conjunction and disjunction to form expressions in propositional logic.

In [19]:
P = Var("P", BoolType)
Q = Var("Q", BoolType)
R = Var("R", BoolType)

print(term.conj(P, Q))
print(term.disj(P, Q))

P & Q
P | Q


Here, we use ASCII characters `&` to represent conjunction, and `|` to represent disjunction. This is in keeping with programming languages such as C, where `&&` stands for `and` and `||` stands for `or`. However, if the user interface allows, we can also print in unicode:

In [20]:
# TODO: add unicode printing.

Here are some more examples. Observe the order of evaluation between conjunction and disjunction, as well as the fact that both logical operators associate to the right.

In [21]:
print(term.conj(P, term.disj(Q, R)))
print(term.disj(term.conj(P, Q), R))
print(term.conj(P, term.conj(Q, R)))
print(term.conj(term.conj(P, Q), R))

P & (Q | R)
P & Q | R
P & Q & R
(P & Q) & R


We do not provide infix operators for conjunction and disjunction. However, we do provide functions And, Or, Implies, and Not to construct propositional formulas. The first three functions take arbitrarily many inputs:

In [22]:
print(And(P, Q, R))
print(Or(P, Q, R))
print(Implies(P, Q, R))
print(Implies(Not(P), Not(Q)))

P & Q & R
P | Q | R
P --> Q --> R
~P --> ~Q


The equality operator is special in that it can take arguments of any type, as long as the two arguments have the same type. The output type is always bool. Hence, the type of the equality operator is $'a \To\ 'a \To bool$. Here $'a$ is a *type variable*, introduced in the previous section. To form an equality expression, we use the `Eq` function. Note how the type of the equality constant is instantiated according to the type of the arguments.

In [23]:
x = Var("x", NatType)
y = Var("y", NatType)
t = Eq(x, y)
print(t)
print(repr(t))

x = y
Comb(Comb(Const(equals, nat => nat => bool), Var(x, nat)), Var(y, nat))


In [24]:
P = Var("P", BoolType)
Q = Var("Q", BoolType)
t = Eq(P, Q)
print(t)
print(repr(t))

P <--> Q
Comb(Comb(Const(equals, bool => bool => bool), Var(P, bool)), Var(Q, bool))


Also note that equality between boolean terms is printed as an if-and-only-if ($\longleftrightarrow$) sign. However, both forms of equalities are represented by constant `equals`.

## Accessing part of a term

For both variables and constants, `name` is the name of the term:

In [25]:
x = Var("x", NatType)
print(repr(x.name))

'x'


Given a combination, `fun` and `arg` returns the function and argument of the combination:

In [26]:
f = Var("f", TFun(NatType, NatType))
t = f(x)
print(t.fun)
print(t.arg)

f
x


Let's try this on the evaluation of a binary operator:

In [27]:
y = Var("y", NatType)
t = x + y
print(t.fun)
print(t.arg)

plus x
y


What's going on here? Recall that a binary operator like `plus` is represented by a function of type $nat \To nat \To nat$. The term $x + y$ is really `plus x y` or, with parenthesis put in, `(plus x) y`. Hence, at the outermost level, the term is a function application with function `plus x` and argument `y`. The term `plus x` is a *partial application* of the function `plus`. It is, on its own, a function of type $nat \To nat$ that adds $x$ to its argument.

Given the application of a binary operator, most often we want to access the operator itself, as well as the two arguments. We have already seen that the second argument can be accessed by `arg`. The first argument can be accessed by `arg1`, and the operator can be accessed by `head`:

In [28]:
print(t.head)
print(t.arg1)

(plus::nat => nat => nat)
x


More generally, given a term of the form $f\ t_1\ t_2\cdots t_n$, the method `strip_comb` returns the pair $f, [t_1, \dots, t_n]$. The properties `head` and `args` correspond to $f$ and $[t_1, \dots, t_n]$, respectively. We demonstrate these functions below.

In [29]:
g = Var("g", TFun(NatType, NatType, NatType))
z = Var("z", NatType)
t = g(x, y, z)
print(t.strip_comb())
print(repr(t.head))
print(t.args)

(Var(g, nat => nat => nat), [Var(x, nat), Var(y, nat), Var(z, nat)])
Var(g, nat => nat => nat)
[Var(x, nat), Var(y, nat), Var(z, nat)]


Python's multiple assignment can be used with `strip_comb` to extract head and arguments at the same time:

In [30]:
f, (a1, a2, a3) = t.strip_comb()
print('f = %s, a1 = %s, a2 = %s, a3 = %s' % (f, a1, a2, a3))

f = g, a1 = x, a2 = y, a3 = z


Given an equality term, its left and right side can be accessed using `arg1` and `arg`. However, writing the code in this way is unintuitive. We further provide properties `lhs` and `rhs` to access the left and right sides of an equality:

In [31]:
t = Eq(x, y)
print(t.lhs)
print(t.rhs)

x
y
